# Assignment 2: Milestone I Natural Language Processing
## Task 1. Basic Text Pre-processing
#### Student Name: Denzel Tan
#### Student ID: s3900098

Date: 19/9/2021

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction


### The Dataset

The Data we are working with is Job Advertisements.

There are approximately 55449 job ads we have to process.

Each job ad comes in the form of:
1. Title
2. WebIndex
3. Company
4. Description

Each job ad is also stored in its respective folder indicitive of the jobs category.

There are 8 categories:
1. Accounting_Finance
2. Engineering
3. Healthcare_Nursing
4. Hospitality_Catering
5. IT
6. PR_Advertising_Marketing
7. Sales
8. Teaching

## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import numpy as np
import re
from sklearn.datasets import load_files
from nltk import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from itertools import chain
from nltk.probability import *
from nltk.util import ngrams

### 1. Examining and loading data

We will:
- Examine the data folder, including the categories and job advertisment txt documents
- Load the data into proper data structures and get it ready for processing.
- Extract webIndex and description into proper data structures.

There are 8 folders in total, each representing a category of job listings:
- Accounting_Finance
- Engineering
- Healthcare_Nursing
- Hospitality_Catering
- IT
- PR_Advertising_Marketing
- Sales
- Teaching

Inside the folder are text files, containing the:
- Title
- WebIndex
- Company
- Description

#### 1.1 Looking at the Data

In [2]:
# Code to inspect the provided data file...
job_data =load_files(r'data')

In [3]:
# This is the structure of the dictionary that load_files gives us
# We have 5 sections
[print(i) for i in job_data]

data
filenames
target_names
target
DESCR


[None, None, None, None, None]

In [4]:
job_data['filenames']

array(['data/Engineering/Job_14624.txt',
       'data/Healthcare_Nursing/Job_31567.txt',
       'data/Hospitality_Catering/Job_50131.txt', ...,
       'data/IT/Job_13401.txt',
       'data/PR_Advertising_Marketing/Job_52696.txt',
       'data/Accounting_Finance/Job_25296.txt'], dtype='<U43')

In [5]:
set(job_data['target'])

{0, 1, 2, 3, 4, 5, 6, 7}

In [6]:
# This means there are 8 folders, each number allocated to the respective category.
job_data['target_names']

['Accounting_Finance',
 'Engineering',
 'Healthcare_Nursing',
 'Hospitality_Catering',
 'IT',
 'PR_Advertising_Marketing',
 'Sales',
 'Teaching']

In [7]:
job_data['target'][:10]

array([1, 2, 3, 2, 7, 2, 2, 5, 4, 6])

In [8]:
# Defining it to variables for easier calling
data, catg = job_data['data'], job_data['target']
num = 5
#print(data[num])
catg[num]

2

In [9]:
# so the Lengths are the same at 55449, which is good.
print(len(data))
print(len(catg))

55449
55449


In [10]:
data[0]

b"Title: Plant Engineer\nWebindex: 62119057\nCompany: W5 Recruitment\nDescription: Our client has established itself as a leading manufacturer and supplier of quality water treatment plants, ranging from basic water softeners and reverse osmosis equipment to customer specified complex water treatment solutions. The company are able to meet their clients' requirements through flexibility in tailoring their product to their needs and budgets. Due to expansion and an increased workload they are seeking to recruit a Planet Engineer to cover accounts along the M4 Corridor Responsibilities will include conducting the routine sampling and analysis of water systems, interpreting results, maintenance and the installation of chemical dosing systems. Servicing accounts within both the industrial and commercial industries the successful candidate will complete all work in accordance to the approved code of practice. The ideal applicant for this position will have a minimum of two years relevant in

#### 1.2 Extracting the desired information

Lets extract the Description and WebIndex now

In [11]:
# Converting the items in the list to a string
# because we need to extract parts with regex, and that requires a string.
data_str = [str(file) for file in data]
type(data_str[0])

str

Now we need to create a list of just the descriptions.

In [12]:
# Extract only the description
des_data = [re.findall(r'Description: (.*)',job) for job in data_str]
des_data[1]


["Timeout Children s Homes Ltd are a rapidly expanding company at the forefront of therapeutic care for young people aged 1017 years, who have experienced emotional behavioural difficulties in their lives. We are looking to recruit Residential Care Workers to be based in our homes in the Swindon area. The successful candidates will need to work collaboratively and cooperatively with other Timeout staff, young people and external agencies. They will also be required to work in consultation with families, social workers, and YOT s and any other professionals involved with the young person including the education team, to deliver effective educational programmes. Successful applicants are required to provide an enhanced disclosure. Disclosure expense will be met by employer. To apply click on the apply button where you will be redirected to our site to complete an application form.'"]

In [13]:
# Flatten the list of (single) lists to now a
# list of strings (the job description)
descr = []
for lis in des_data:
    for job_des in lis:
        descr.append(job_des)
print(len(descr))
type(descr[0])

55449


str

Yup looks correct. We now have a list of strings, each string being the job description.

Done. Lets move on to tokenization.

### 2. Pre-processing data
Perform the required text pre-processing steps.

...... Sections and code blocks on basic text pre-processing


<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

#### 2.1 Tokenize each Job Description

Creating our Tokenize Function:

In [14]:
from nltk import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from itertools import chain


def tokenizeReview(review):
    """
        This function first convert all words to lowercases, 
        it then segment the raw review into sentences and tokenize each sentences 
        and convert the review to a list of tokens.
    """        
    #review = raw_review.decode('utf-8') # convert the bytes-like object to python string, need this before we apply any pattern search on it
    nl_review = review.lower() # cover all words to lowercase
    
    # segament into sentences
    sentences = sent_tokenize(nl_review)
    
    # tokenize each sentence
    pattern = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"
    tokenizer = RegexpTokenizer(pattern) 
    token_lists = [tokenizer.tokenize(sen) for sen in sentences]
    
    # merge them into a list of tokens
    tokenised_review = list(chain.from_iterable(token_lists))
    return tokenised_review

Now lets create a a function which shows some general statistics:

In [15]:
# Creating function for basic statistics
def stats_print(tk_reviews):
    words = list(chain.from_iterable(tk_reviews)) # we put all the tokens in the corpus in a single list
    vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words
    lexical_diversity = len(vocab)/len(words)
    print("Vocabulary size: ",len(vocab))
    print("Total number of tokens: ", len(words))
    print("Lexical diversity: ", lexical_diversity)
    print("Total number of reviews:", len(tk_reviews))
    lens = [len(article) for article in tk_reviews]
    print("Average review length:", np.mean(lens))
    print("Maximun review length:", np.max(lens))
    print("Minimun review length:", np.min(lens))
    print("Standard deviation of review length:", np.std(lens))

In [16]:
# Checking what it looks like
descr[0]

'Our client has established itself as a leading manufacturer and supplier of quality water treatment plants, ranging from basic water softeners and reverse osmosis equipment to customer specified complex water treatment solutions. The company are able to meet their clients\' requirements through flexibility in tailoring their product to their needs and budgets. Due to expansion and an increased workload they are seeking to recruit a Planet Engineer to cover accounts along the M4 Corridor Responsibilities will include conducting the routine sampling and analysis of water systems, interpreting results, maintenance and the installation of chemical dosing systems. Servicing accounts within both the industrial and commercial industries the successful candidate will complete all work in accordance to the approved code of practice. The ideal applicant for this position will have a minimum of two years relevant industry experience and will have knowledge of reverse osmosis, water softeners, wa

In [17]:
# TOKENIZING every job description
tk_descr = [tokenizeReview(des) for des in descr]

In [18]:
stats_print(tk_descr)

Vocabulary size:  89970
Total number of tokens:  13935165
Lexical diversity:  0.006456328288900777
Total number of reviews: 55449
Average review length: 251.3149921549532
Maximun review length: 2163
Minimun review length: 10
Standard deviation of review length: 127.54164999587417


In [19]:
len(tk_descr)

55449

In [20]:
tk_descr[0]

['our',
 'client',
 'has',
 'established',
 'itself',
 'as',
 'a',
 'leading',
 'manufacturer',
 'and',
 'supplier',
 'of',
 'quality',
 'water',
 'treatment',
 'plants',
 'ranging',
 'from',
 'basic',
 'water',
 'softeners',
 'and',
 'reverse',
 'osmosis',
 'equipment',
 'to',
 'customer',
 'specified',
 'complex',
 'water',
 'treatment',
 'solutions',
 'the',
 'company',
 'are',
 'able',
 'to',
 'meet',
 'their',
 'clients',
 'requirements',
 'through',
 'flexibility',
 'in',
 'tailoring',
 'their',
 'product',
 'to',
 'their',
 'needs',
 'and',
 'budgets',
 'due',
 'to',
 'expansion',
 'and',
 'an',
 'increased',
 'workload',
 'they',
 'are',
 'seeking',
 'to',
 'recruit',
 'a',
 'planet',
 'engineer',
 'to',
 'cover',
 'accounts',
 'along',
 'the',
 'm',
 'corridor',
 'responsibilities',
 'will',
 'include',
 'conducting',
 'the',
 'routine',
 'sampling',
 'and',
 'analysis',
 'of',
 'water',
 'systems',
 'interpreting',
 'results',
 'maintenance',
 'and',
 'the',
 'installation',


Nice. So now we have tokenized every job description

#### 2.2 Remove words with <2 Length

In [21]:
# Testing with the first one
for word in tk_descr[0]:
    if len(word)<2:
        print(word)

a
a
m
a
a
a


In [22]:
# Removing the words with loop
tk_descr = [[word for word in des if len(word) >= 2] for des in tk_descr]

In [23]:
# test again
for word in tk_descr[0]:
    if len(word)<2:
        print(word)

Great it doesn't return anything. We got rid of them all.

#### 2.3 Removing Stopwords

In [24]:
# Extracting the stopwords from the .txt file and making sure the '\n' isnt included
with open('stopwords_en.txt','r') as f:
    lines = f.read()
    stopwords = lines.splitlines()
stopwords


['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 "c'mon",
 "c's",
 'came',
 'can',
 "can't",
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'c

Ok stopwords extracted into list. Lets get to removing them now.

In [25]:
# Finding the words that would be kept when we delete using the stopwords list
[word for word in tk_descr[0] if word not in stopwords]

['client',
 'established',
 'leading',
 'manufacturer',
 'supplier',
 'quality',
 'water',
 'treatment',
 'plants',
 'ranging',
 'basic',
 'water',
 'softeners',
 'reverse',
 'osmosis',
 'equipment',
 'customer',
 'complex',
 'water',
 'treatment',
 'solutions',
 'company',
 'meet',
 'clients',
 'requirements',
 'flexibility',
 'tailoring',
 'product',
 'budgets',
 'due',
 'expansion',
 'increased',
 'workload',
 'seeking',
 'recruit',
 'planet',
 'engineer',
 'cover',
 'accounts',
 'corridor',
 'responsibilities',
 'include',
 'conducting',
 'routine',
 'sampling',
 'analysis',
 'water',
 'systems',
 'interpreting',
 'results',
 'maintenance',
 'installation',
 'chemical',
 'dosing',
 'systems',
 'servicing',
 'accounts',
 'industrial',
 'commercial',
 'industries',
 'successful',
 'candidate',
 'complete',
 'work',
 'accordance',
 'approved',
 'code',
 'practice',
 'ideal',
 'applicant',
 'position',
 'minimum',
 'years',
 'relevant',
 'industry',
 'experience',
 'knowledge',
 'rever

In [26]:
len(tk_descr[0])

167

In [27]:
# Now REMOVING ALL stopwords
set_stopwords = set(stopwords)
tk_descr = [[word for word in des if word not in set_stopwords] for des in tk_descr]

In [28]:
# Looks like the length decreased. The stopword removal worked.
len(tk_descr[0])

99

In [29]:
stats_print(tk_descr)

Vocabulary size:  89431
Total number of tokens:  7931546
Lexical diversity:  0.011275355397295812
Total number of reviews: 55449
Average review length: 143.04218290681527
Maximun review length: 1280
Minimun review length: 7
Standard deviation of review length: 74.94886837345045


Also interesting note: The stopwords list actually has a duplicate in it, as found when the set was 1 less than the normal list. The duplicate is "would". However, for stopword removal, a list or set shouldn't impact the final product, so a list was used to be safe. 

Note that if the list was much bigger, a set might have needed to be used for speed

In [30]:
kid = FreqDist(stopwords)

In [31]:
kid.most_common()

[('would', 2),
 ('a', 1),
 ("a's", 1),
 ('able', 1),
 ('about', 1),
 ('above', 1),
 ('according', 1),
 ('accordingly', 1),
 ('across', 1),
 ('actually', 1),
 ('after', 1),
 ('afterwards', 1),
 ('again', 1),
 ('against', 1),
 ("ain't", 1),
 ('all', 1),
 ('allow', 1),
 ('allows', 1),
 ('almost', 1),
 ('alone', 1),
 ('along', 1),
 ('already', 1),
 ('also', 1),
 ('although', 1),
 ('always', 1),
 ('am', 1),
 ('among', 1),
 ('amongst', 1),
 ('an', 1),
 ('and', 1),
 ('another', 1),
 ('any', 1),
 ('anybody', 1),
 ('anyhow', 1),
 ('anyone', 1),
 ('anything', 1),
 ('anyway', 1),
 ('anyways', 1),
 ('anywhere', 1),
 ('apart', 1),
 ('appear', 1),
 ('appreciate', 1),
 ('appropriate', 1),
 ('are', 1),
 ("aren't", 1),
 ('around', 1),
 ('as', 1),
 ('aside', 1),
 ('ask', 1),
 ('asking', 1),
 ('associated', 1),
 ('at', 1),
 ('available', 1),
 ('away', 1),
 ('awfully', 1),
 ('b', 1),
 ('be', 1),
 ('became', 1),
 ('because', 1),
 ('become', 1),
 ('becomes', 1),
 ('becoming', 1),
 ('been', 1),
 ('before', 1

#### 2.4 Remove words that appear once in the document collection

In [32]:
words = list(chain.from_iterable(tk_descr)) # we put all the tokens in the corpus in a single list
vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words

In [33]:
term_fd = FreqDist(words)

In [34]:
# Checking the last 3 on the most common list
term_fd.most_common()[-3:]

[('automationtestanalystqatesterseleniumwebdriverc', 1),
 ('timecales', 1),
 ('dutchpayroller', 1)]

In [35]:
# Appending all words that occur once to a list.
occur_once = []
occur = 1
for term, occurances in term_fd.items(): # Since FreqDist gives a dictionary, we need to search each key:value and 
    if occurances == occur: # find values that match to what we want.
        occur_once.append(term)

In [36]:
# Validated. Looks like our list works. 
occur_once[-3:]

['automationtestanalystqatesterseleniumwebdriverc',
 'timecales',
 'dutchpayroller']

In [37]:
len(occur_once)

49237

In [38]:
occur_once_set = set(occur_once)

In [39]:
len(occur_once_set)

49237

Now we have a list of words (single occurance) that we want to remove. Lets follow the same process as the stopwords.

In [40]:
# Now REMOVING ALL words that occur once
tk_descr = [[word for word in des if word not in occur_once_set] for des in tk_descr]

In [41]:
len(tk_descr)

55449

In [42]:
stats_print(tk_descr)

Vocabulary size:  40194
Total number of tokens:  7882309
Lexical diversity:  0.0050992672324822585
Total number of reviews: 55449
Average review length: 142.15421378203393
Maximun review length: 1269
Minimun review length: 7
Standard deviation of review length: 74.62098165848764


In [43]:
89431-49237

40194

Nice. We got rid of the single occurance words, validated by our simple math.

Note: A set needed to be used (not that it changed anything about the occur once list) but it was significently faster.

#### 2.5 Remove the top 50 most frequent words based on Document Frequency

In [44]:
# Grabbing the top 50 most common words based on document freq
words_2 = list(chain.from_iterable([set(des) for des in tk_descr]))
doc_fd = FreqDist(words_2)  # compute document frequency for each unique word/type
top_50 = doc_fd.most_common(50)
top_50

[('experience', 43643),
 ('role', 34680),
 ('work', 33683),
 ('team', 32585),
 ('working', 30713),
 ('skills', 30412),
 ('client', 26899),
 ('job', 25552),
 ('business', 24739),
 ('uk', 24133),
 ('excellent', 22978),
 ('opportunity', 22678),
 ('company', 22263),
 ('management', 20620),
 ('required', 20555),
 ('development', 20223),
 ('apply', 20125),
 ('based', 19333),
 ('successful', 19118),
 ('join', 18682),
 ('www', 18421),
 ('salary', 18402),
 ('cv', 18383),
 ('support', 18286),
 ('knowledge', 17844),
 ('strong', 16472),
 ('environment', 16408),
 ('posted', 16398),
 ('jobseeking', 16342),
 ('candidate', 16304),
 ('originally', 16294),
 ('leading', 16194),
 ('high', 15921),
 ('service', 15623),
 ('manager', 15587),
 ('good', 15249),
 ('ability', 15152),
 ('including', 14857),
 ('position', 14564),
 ('services', 14501),
 ('benefits', 14434),
 ('training', 14217),
 ('essential', 13915),
 ('experienced', 13826),
 ('key', 13567),
 ('contact', 13551),
 ('level', 13522),
 ('recruitment', 

In [45]:
# Testing to see if the term frequency would be different
test = list(chain.from_iterable([des for des in tk_descr]))
doc_fd2 = FreqDist(test)
doc_fd2.most_common(5)

[('experience', 104058),
 ('role', 66536),
 ('work', 64579),
 ('team', 62304),
 ('business', 59182)]

Yup we can clearly see that there is a huge difference using the document frequency instead. We have now validated it is working.

In [46]:
# Test to see if list is correct
top_50[0][0]

'experience'

In [47]:
# Length looks correct
len(top_50)

50

In [48]:
# Making a list of the top 50 words itself
top_50_list = []
for i in range(0,len(top_50)):
    top_50_list.append(top_50[i][0])
top_50_list

['experience',
 'role',
 'work',
 'team',
 'working',
 'skills',
 'client',
 'job',
 'business',
 'uk',
 'excellent',
 'opportunity',
 'company',
 'management',
 'required',
 'development',
 'apply',
 'based',
 'successful',
 'join',
 'www',
 'salary',
 'cv',
 'support',
 'knowledge',
 'strong',
 'environment',
 'posted',
 'jobseeking',
 'candidate',
 'originally',
 'leading',
 'high',
 'service',
 'manager',
 'good',
 'ability',
 'including',
 'position',
 'services',
 'benefits',
 'training',
 'essential',
 'experienced',
 'key',
 'contact',
 'level',
 'recruitment',
 'candidates',
 'provide']

In [49]:
# Initial length check
len(tk_descr[0])

99

In [50]:
# Removing the top 50 words now
top_50_list_set = set(top_50_list)
tk_descr = [[word for word in des if word not in top_50_list_set] for des in tk_descr]

In [51]:
# Validated. Looks like words got removed.
len(tk_descr[0])

84

In [52]:
stats_print(tk_descr)

Vocabulary size:  40144
Total number of tokens:  6307205
Lexical diversity:  0.006364784401331493
Total number of reviews: 55449
Average review length: 113.74785839239662
Maximun review length: 1138
Minimun review length: 4
Standard deviation of review length: 63.06833304768886


Hooray. We removed the top 50 most frequent words from every description.

#### 2.6 Extract the Top 10 Bigrams based on term frequency (save to .txt file)

In [53]:
# Updating the words and vocab to latest.
words = list(chain.from_iterable(tk_descr)) # we put all the tokens in the corpus in a single list
vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words

In [54]:
# Setting up our bigrams
bigrams = ngrams(words, n =2)
fdbigram = FreqDist(bigrams)

In [55]:
# Finding top 10 most common Bigrams
top_10 = fdbigram.most_common(10)
top_10

[(('xe', 'xa'), 22105),
 (('employment', 'agency'), 8055),
 (('track', 'record'), 5472),
 (('acting', 'employment'), 5095),
 (('sql', 'server'), 4804),
 (('asp', 'net'), 4687),
 (('relation', 'vacancy'), 3977),
 (('sales', 'executive'), 3619),
 (('chef', 'de'), 3586),
 (('nursing', 'home'), 3503)]

In [56]:
# Making list of bigrams, combined into string with its count.
top_10_combined = []
for i in range(0,len(top_10)):
    top_10_combined.append( " ".join(top_10[i][0]) + ',' + str(top_10[i][1]))

In [57]:
top_10_combined

['xe xa,22105',
 'employment agency,8055',
 'track record,5472',
 'acting employment,5095',
 'sql server,4804',
 'asp net,4687',
 'relation vacancy,3977',
 'sales executive,3619',
 'chef de,3586',
 'nursing home,3503']

Ok we got it into a list. Now we just have to save it later.

In [58]:
stats_print(tk_descr)

Vocabulary size:  40144
Total number of tokens:  6307205
Lexical diversity:  0.006364784401331493
Total number of reviews: 55449
Average review length: 113.74785839239662
Maximun review length: 1138
Minimun review length: 4
Standard deviation of review length: 63.06833304768886


#### 2.7 Build Vocab list and cleaned Job Ad list

In [59]:
# Updating the words and vocab to latest.
words = list(chain.from_iterable(tk_descr)) # we put all the tokens in the corpus in a single list
vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words
vocab_list = list(vocab)

In [60]:
len(vocab_list)

40144

In [61]:
# Sorting our list into alphabetical order
vocab_list = sorted(vocab_list)
vocab_list

["a'level",
 'aa',
 'aaa',
 'aaappointments',
 'aab',
 'aac',
 'aacc',
 'aae',
 'aah',
 'aak',
 'aamm',
 'aan',
 'aand',
 'aantrekkelijk',
 'aap',
 'aar',
 'aarca',
 'aardman',
 'aareon',
 'aaron',
 'aaronwallis',
 'aarosette',
 'aarosettehotel',
 'aarosettelivein',
 'aarosetterestaurant',
 'aarosetterestaurantlondon',
 'aarosettes',
 'aarosettesinternationalchainofhotels',
 'aarosetteslivein',
 'aasl',
 'aasp',
 'aastra',
 'aat',
 'aatom',
 'ab',
 'aba',
 'abacus',
 'abaility',
 'abandoned',
 'abandonment',
 'abandons',
 'abap',
 'abaqus',
 'abatement',
 'abb',
 'abbas',
 'abbey',
 'abbeywood',
 'abbie',
 'abbot',
 'abbots',
 'abbott',
 'abby',
 'abbyy',
 'abc',
 'abd',
 'abdo',
 'abdominal',
 'abdul',
 'abdulla',
 'abel',
 'abellprotocoleducation',
 'abenefit',
 'aberdare',
 'aberdeen',
 "aberdeen's",
 'aberdeenshire',
 'aberdenshire',
 'aberfeldy',
 'abergavenny',
 'abertillery',
 'aberystwyth',
 'abfa',
 'abgeschlossenes',
 'abi',
 'abid',
 'abide',
 'abiding',
 'abigail',
 'abili'

In [62]:
# Testing if correct
vocab_list[0]

"a'level"

In [63]:
# Saving the vocab and its Index to a list in the desired format.
full_vocab = []
for i in range(0,len(vocab_list)):
    full_vocab.append(vocab_list[i] +":"+ str(i))

In [64]:
# Testing to compare below if our list worked and appending the correct form.
i = 21
vocab_list[i] +":"+ str(i)

'aarosette:21'

In [65]:
full_vocab

["a'level:0",
 'aa:1',
 'aaa:2',
 'aaappointments:3',
 'aab:4',
 'aac:5',
 'aacc:6',
 'aae:7',
 'aah:8',
 'aak:9',
 'aamm:10',
 'aan:11',
 'aand:12',
 'aantrekkelijk:13',
 'aap:14',
 'aar:15',
 'aarca:16',
 'aardman:17',
 'aareon:18',
 'aaron:19',
 'aaronwallis:20',
 'aarosette:21',
 'aarosettehotel:22',
 'aarosettelivein:23',
 'aarosetterestaurant:24',
 'aarosetterestaurantlondon:25',
 'aarosettes:26',
 'aarosettesinternationalchainofhotels:27',
 'aarosetteslivein:28',
 'aasl:29',
 'aasp:30',
 'aastra:31',
 'aat:32',
 'aatom:33',
 'ab:34',
 'aba:35',
 'abacus:36',
 'abaility:37',
 'abandoned:38',
 'abandonment:39',
 'abandons:40',
 'abap:41',
 'abaqus:42',
 'abatement:43',
 'abb:44',
 'abbas:45',
 'abbey:46',
 'abbeywood:47',
 'abbie:48',
 'abbot:49',
 'abbots:50',
 'abbott:51',
 'abby:52',
 'abbyy:53',
 'abc:54',
 'abd:55',
 'abdo:56',
 'abdominal:57',
 'abdul:58',
 'abdulla:59',
 'abel:60',
 'abellprotocoleducation:61',
 'abenefit:62',
 'aberdare:63',
 'aberdeen:64',
 "aberdeen's:65

## Saving required outputs
Save the vocabulary, bigrams and job advertisment txt as per spectification.
- vocab.txt
- bigram.txt
- job_ads.txt

#### Saving Bigrams

In [66]:
# Saving Bigrams Function
def save_bigrams(bigramFilename,bigrams):
    out_file = open(bigramFilename, 'w') # creates a txt file and open to save Bigrams
    string = "\n".join([str(s) for s in bigrams])
    out_file.write(string)
    out_file.close() # close the file 

In [67]:
# Saving Bigrams
save_bigrams('bigram.txt',top_10_combined)

#### Saving Vocab

Lets just use the same function as the Bigrams, since it does exactly the same thing.

And to save time and be effecient! No point reinventing the wheel right?



In [68]:
# Saving Vocab
save_bigrams('vocab.txt',full_vocab)


### Saving job advertisements

We have 5 sections required for our output. Lets put them each into a list first.

#### 1. Id

In [69]:
job_data['filenames'][0]

'data/Engineering/Job_14624.txt'

In [70]:
# Extracting the 5 digit ID from each filename
id_list = []
for i in range(0,len(job_data['filenames'])):
    id_list.append(re.findall(r'_(\d{5}).txt',job_data['filenames'][i]))
len(id_list)

55449

In [71]:
# Flatten the list of lists
id_list_flat = list(chain.from_iterable(id_list))
id_list_flat

['14624',
 '31567',
 '50131',
 '31419',
 '47238',
 '36205',
 '30175',
 '54070',
 '05781',
 '44021',
 '02465',
 '13082',
 '36404',
 '13998',
 '51419',
 '07060',
 '18399',
 '39194',
 '44876',
 '47390',
 '08890',
 '02696',
 '00468',
 '39671',
 '52467',
 '31620',
 '32012',
 '06821',
 '13089',
 '05164',
 '01998',
 '20455',
 '46705',
 '15154',
 '50111',
 '52754',
 '09944',
 '18991',
 '10976',
 '45145',
 '30000',
 '09713',
 '50285',
 '43066',
 '43015',
 '50205',
 '49829',
 '05736',
 '13136',
 '51362',
 '09825',
 '31939',
 '30884',
 '14843',
 '42277',
 '31834',
 '28281',
 '35064',
 '53016',
 '06594',
 '46647',
 '23599',
 '53176',
 '20842',
 '54347',
 '21895',
 '04243',
 '04431',
 '39265',
 '04778',
 '28251',
 '00205',
 '25094',
 '21655',
 '54708',
 '11778',
 '19802',
 '15400',
 '47553',
 '35715',
 '29910',
 '11775',
 '12634',
 '20517',
 '01946',
 '40929',
 '28502',
 '11197',
 '21182',
 '25544',
 '04297',
 '04836',
 '47960',
 '18773',
 '43268',
 '18032',
 '28013',
 '00835',
 '22435',
 '18149',


#### 2. Category

In [72]:
# Use regex to pull out the category from filename
cat_list = []
for i in range(0,len(job_data['filenames'])):
    cat_list.append(re.findall(r'/(\w+)/',job_data['filenames'][i]))
len(cat_list)

55449

In [73]:
# Flatten list of lists into just a list, then find the set to confirm if all Categories were picked up.
cat_list_flat = list(chain.from_iterable(cat_list))
set(cat_list_flat)

{'Accounting_Finance',
 'Engineering',
 'Healthcare_Nursing',
 'Hospitality_Catering',
 'IT',
 'PR_Advertising_Marketing',
 'Sales',
 'Teaching'}

Looks like we got all categories captured. That verifies it for us now.

#### 3. WebIndex

In [74]:
# Making list of web index's 
web_list = []
for i in range(0,len(job_data["data"])):
    web_list.append(re.findall(r'Webindex: (\d{8})',str(job_data["data"][i])))
len(web_list)

55449

In [75]:
# Flatten the list of lists
web_list_flat = list(chain.from_iterable(web_list))
web_list_flat

['62119057',
 '66314490',
 '69182387',
 '64752715',
 '71792012',
 '70612741',
 '46628810',
 '69172722',
 '68807434',
 '72479636',
 '68092333',
 '72229719',
 '70763910',
 '72578110',
 '70759896',
 '69159371',
 '69560135',
 '66886072',
 '68608673',
 '71876246',
 '70090376',
 '68181520',
 '64558781',
 '67896016',
 '72234849',
 '66426156',
 '66952145',
 '69043129',
 '72230002',
 '68691070',
 '67770759',
 '71357143',
 '70761594',
 '66507219',
 '69182211',
 '66543099',
 '70783118',
 '69868209',
 '71444351',
 '68710342',
 '46626860',
 '70780874',
 '69470916',
 '71482344',
 '71435250',
 '69183409',
 '68834497',
 '68806742',
 '72239601',
 '70735413',
 '70782077',
 '66932685',
 '51061170',
 '65184188',
 '70078924',
 '66626362',
 '71290919',
 '69689660',
 '67755530',
 '69039153',
 '70761124',
 '67381906',
 '68215155',
 '71619334',
 '69763259',
 '72439441',
 '68627615',
 '68628765',
 '66930750',
 '68677600',
 '71226442',
 '58966613',
 '68686262',
 '72248299',
 '70682968',
 '71686197',
 '70607592',

#### 4. Title

In [76]:
# The Regex Pattern for extracting the Title
re.findall(r'Title: (.+)\\nWebindex:',str(job_data["data"][21]))

['C Developer  Top Asset Management Firm']

In [77]:
# Making list of Titles
title_list = []
for i in range(0,len(job_data["data"])):
    title_list.append(re.findall(r'Title: (.+)\\nWebindex:',str(job_data["data"][i])))
len(title_list)

55449

In [78]:
# Flatten the list of lists
title_list_flat = list(chain.from_iterable(title_list))
title_list_flat

['Plant Engineer',
 'Residential Care Worker',
 'CHEF DE RANG FOR MICHELIN STARRED RESTAURANT',
 'Inpatient Ward Team Leader',
 'Information Services Support Analyst',
 'Physiotherapist  Cambridge',
 'Youth Justice Officer Job Luton',
 'Front End Web Developer  Slough  ****k****k',
 'NET / Web Developer  Clayton West, Huddersfield, West Yorkshire',
 'Senior Account Manager (Telecoms / IT)',
 'IT Project Manager (C/ASPnet) ****K Leicester',
 'Cisco Call Manager Technician  Greater Manchester',
 'Senior Nurse  Small Specialist Care Home',
 'Senior Oracle DBA/Database AdministratorNorth Yorkshire',
 'Cluster Sales Manager  Maternity Cover',
 'Software Build Manager, MSBuild',
 'Electrical Fitter  Crawley ,Sussex',
 'Sales Manager Door Automation',
 'Year ****/6 Teacher, Cannock',
 'Experienced KS****/KS**** Science teacher Braintree',
 'Design Engineer  Software and Firmware (Python)',
 'C Developer  Top Asset Management Firm',
 'Senior Javascript developer',
 'RESEARCHER, BOLTON',
 'Seni

#### 5. Description 

In [79]:
# Joining all description tokens togather into a single string
des_string = [' '.join(token) for token in tk_descr]
len(des_string)

55449

In [80]:
# Now a list of JUST the description
des_string[0:2]

['established manufacturer supplier quality water treatment plants ranging basic water softeners reverse osmosis equipment customer complex water treatment solutions meet clients requirements flexibility tailoring product budgets due expansion increased workload seeking recruit planet engineer cover accounts corridor responsibilities include conducting routine sampling analysis water systems interpreting results maintenance installation chemical dosing systems servicing accounts industrial commercial industries complete accordance approved code practice ideal applicant minimum years relevant industry reverse osmosis water softeners water filters uv equipment full driving license return offering competitive package ideal',
 'timeout children homes rapidly expanding forefront therapeutic care young people aged years emotional behavioural difficulties lives recruit residential care workers homes swindon area collaboratively cooperatively timeout staff young people external agencies consul

In [81]:
# Creating a HUGE list of every single description + its corresponding info. 
all_info = []
for i in range(0,len(job_data["data"])):
    
    string = "ID: " + id_list_flat[i] + "\nCategory: " + cat_list_flat[i] + "\nWebindex: " + web_list_flat[i] + "\nTitle: " + title_list_flat[i] + "\nDescription: " + des_string[i]
    all_info.append(string)
len(all_info)

55449

In [82]:
# Checking our huge list complete with all info
all_info[:2]

['ID: 14624\nCategory: Engineering\nWebindex: 62119057\nTitle: Plant Engineer\nDescription: established manufacturer supplier quality water treatment plants ranging basic water softeners reverse osmosis equipment customer complex water treatment solutions meet clients requirements flexibility tailoring product budgets due expansion increased workload seeking recruit planet engineer cover accounts corridor responsibilities include conducting routine sampling analysis water systems interpreting results maintenance installation chemical dosing systems servicing accounts industrial commercial industries complete accordance approved code practice ideal applicant minimum years relevant industry reverse osmosis water softeners water filters uv equipment full driving license return offering competitive package ideal',
 'ID: 31567\nCategory: Healthcare_Nursing\nWebindex: 66314490\nTitle: Residential Care Worker\nDescription: timeout children homes rapidly expanding forefront therapeutic care yo

### Now we save the data.

In [83]:
def save_all(filename,contents):
    out_file = open(filename, 'w') # creates a txt file and open to save the ads
    out_file.write("\n".join([info for info in contents])) # Write the content
    out_file.close() # close the file

In [84]:
save_all("job_ads.txt",all_info)

## Summary

Overall, we exported 3 files.

The main difficulty was extracting the description out of the original data file, however that was solved via the very useful Regex.

I learnt that Sets can be very useful, and can be significently faster than lists in some applications.

## Couple of notes for all code blocks in this notebook
- please provide proper comment on your code
- Please re-start and run all cells to make sure codes are runable and include your output in the submission.   
<span style="color: red"> This markdown block can be removed once the task is completed. </span>

In [85]:
stats_print(tk_descr)

Vocabulary size:  40144
Total number of tokens:  6307205
Lexical diversity:  0.006364784401331493
Total number of reviews: 55449
Average review length: 113.74785839239662
Maximun review length: 1138
Minimun review length: 4
Standard deviation of review length: 63.06833304768886
